In [ ]:
import pandas as pd
import numpy as np

import contractions

import nltk

from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

import re
from nltk.stem import WordNetLemmatizer

In [ ]:
df = pd.read_json('MitreData.json')

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
#df['combined'] = df.iloc[:, df.columns != 'id']
df['combined'] = df.iloc[:,df.columns != 'id'].apply(lambda x: ",".join(x.astype(str)), axis=1)

In [ ]:
df['combined'][1]

In [ ]:
X_train = df['combined']
y_train = df['id']
train_X =[]

In [ ]:
# import re
# def pre_process(text):
    
#     # lowercase
#     text=text.lower()
    
#     #remove tags
#     text=re.sub("","",text)
    
#     # remove special characters and digits
#     text=re.sub("(\\d|\\W)+"," ",text)
    
#     return text

# X_train = X_train.apply(lambda x:pre_process(x))

In [ ]:
# X_train[1]

In [ ]:
print(len(X_train))

In [ ]:
print(X_train[0])

In [ ]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
stop_words.add('nan')

punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
for sent in X_train:
    for ele in sent:
        if ele in punc:
            sent = contractions.fix(sent)
            sent = sent.replace(ele, " ").strip().lower()
    word_tokens = word_tokenize(sent)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    for fs in range(len(filtered_sentence)):
        filtered_sentence[fs] = porter.stem(filtered_sentence[fs])
    #print(filtered_sentence)
    train_X.append(" ".join(filtered_sentence))

In [ ]:
print(train_X[0])

In [ ]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

In [ ]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

In [ ]:
print(tf_idf.get_feature_names())

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, y_train)

In [ ]:
test = ["Suspicious Process Activity - Potential Mimikatz or Hash Passing Token Creation - Powershell Privileged Service Call Analytic"]

In [ ]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
test_processed = []
for sent in test:
  for ele in sent:
      if ele in punc:
          sent = contractions.fix(sent)
          sent = sent.replace(ele, " ").strip().lower()
  word_tokens = word_tokenize(sent)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  for fs in range(len(filtered_sentence)):
    filtered_sentence[fs] = porter.stem(filtered_sentence[fs])
  #print(filtered_sentence)
  test_processed.append(" ".join(filtered_sentence))

In [ ]:
print(test_processed)

In [ ]:
test_input = tf_idf.transform(test_processed)
test_input.shape

In [ ]:
res=naive_bayes_classifier.predict(test_processed)

In [ ]:
print(res)

In [ ]:
score_train = naive_bayes_classifier.score(X_train_tf, y_train)
print(score_train)